<a href="https://colab.research.google.com/github/gvosper25/GeorgiaVosper_3009ICT/blob/main/GeorgiaVosper_3009ICT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing and Visualization
-----
**Assignment Topic: Data Analysis on Car Advertisements**


Student Name: Georgia Vosper

SNumber: S5223182

# Data Import

In [69]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import sqlite3
import seaborn as sns
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
car_data = pd.read_csv ('/content/drive/MyDrive/3009ICT-Assignment/car_dataset.csv')
sellers_data = pd.read_csv ('/content/drive/MyDrive/3009ICT-Assignment/seller_dataset.csv')

# Data Formatting

## a. Numerical Conversion

In [71]:
cols_to_clean = ['mileage','num_of_doors','seating_capacity','fuel_consumption','price (AUD)']
for col in cols_to_clean:
  car_data[col] = car_data[col].astype(str).str.extract(r'([\d.]+)').astype(float)

Run test

In [72]:
car_data [['mileage','num_of_doors','seating_capacity','fuel_consumption','price (AUD)']].head ()

,mileage,num_of_doors,seating_capacity,fuel_consumption,price (AUD)
0,0.0,2.0,2.0,100.0,10080.99411
1,0.0,5.0,7.0,10.0,161943.68050
2,0.0,5.0,8.0,100.0,35830.03930
3,0.0,5.0,5.0,100.0,30526.38376
4,0.0,5.0,8.0,100.0,34413.03209


## b. Engine Feature Split

In [73]:
car_data[['type_of_engine','engine_capacity']] =car_data ['engine'].str.extract(r'(\w+)\s*[\t]*\s*([\d.]+)\s*L')
car_data['engine_capacity'] =car_data ['engine_capacity'].astype(float)

Run test

In [74]:
car_data[['engine','type_of_engine','engine_capacity']].head()

,engine,type_of_engine,engine_capacity
0,Petrol\t1.0 L,Petrol,1.0
1,Petrol\t3.4 L,Petrol,3.4
2,Petrol\t2.0 L,Petrol,2.0
3,Petrol\t1.8 L,Petrol,1.8
4,Petrol\t2.0 L,Petrol,2.0


## c. Standardising Text Data

In [75]:
cols_to_lower = ['origin','condition','car_model','exterior_color','interior_color','type_of_engine','fuel_system','transmission','drive_type','brand','grade']
for col in cols_to_lower:
  car_data[col] = car_data[col].astype(str).str.lower()

Run test

In [76]:
car_data[['origin','condition','car_model','exterior_color','interior_color','type_of_engine','fuel_system','transmission','drive_type','brand','grade']].head()

,origin,condition,car_model,exterior_color,interior_color,type_of_engine,fuel_system,transmission,drive_type,brand,grade
0,domestic assembly,new car,truck,white,gray,petrol,nan,manual,rfd - rear-wheel drive,suzuki,super carry truck
1,imported,new car,suv,black,black,petrol,nan,automatic,awd - 4-wheel drive (awd),toyota,land cruiser
2,domestic assembly,new car,crossover,silver,brown,petrol,nan,automatic,rfd - rear-wheel drive,toyota,innova
3,imported,new car,suv,white,black,petrol,nan,automatic,fwd - front-wheel drive,toyota,corolla cross
4,domestic assembly,new car,crossover,silver,gray,petrol,nan,automatic,rfd - rear-wheel drive,toyota,innova


## d. Numerical Scaling

In [77]:
columns_to_scale = ['engine_capacity','num_of_doors','seating_capacity','fuel_consumption','price (AUD)']

scaler = MinMaxScaler()

import re

for col in columns_to_scale:
  clean_name = re.sub(r'[^a-zA-Z0-9]','_',col).lower()
  new_col = f'normalised_{clean_name}'
  car_data[new_col] = scaler.fit_transform(car_data[[col]])

Run test

In [78]:
car_data[['normalised_engine_capacity','normalised_price__aud_']].describe()

,normalised_engine_capacity,normalised_price__aud_
count,29285.000000,30652.000000
mean,0.154511,0.019597
std,0.064822,0.035946
min,0.000000,0.000000
25%,0.111111,0.007500
50%,0.150794,0.011407
75%,0.182540,0.018500
max,1.000000,1.000000


Save as a new CSV file

In [79]:
car_data.to_csv('/content/drive/MyDrive/3009ICT-Assignment/processed_car_dataset.csv', index = False)

# Data Exploration and Cleaning

## a. Statistics Computation

In [80]:
import numpy as np

columns = ['price (AUD)','mileage','year_of_manufacture']

for col in columns:
  print(f"\n---{col}---")
  print("Mean:", car_data[col].mean())
  print("Median:", car_data[col].median())
  print("Mode", car_data[col].mode().iloc[0])
  print("Standard Deviation:", car_data[col].std())
  print("Range", car_data[col].max()-car_data[col].min())


---price (AUD)---
Mean: 42843.16554304759
Median: 24939.32679
Mode 40485.92011
Standard Deviation: 78585.58399261319
Range 2186239.6858785423

---mileage---
Mean: 42.11447866370873
Median: 20.0
Mode 0.0
Standard Deviation: 74.87235716224309
Range 999.0

---year_of_manufacture---
Mean: 2017.320346178968
Median: 2019.0
Mode 2023.0
Standard Deviation: 5.3317724314591874
Range 33.0


## b. Outliers and Missing Values

### i) Make a copy of the data before cleaning

In [81]:
before_cleaning_rows = car_data.shape[0]

numeric_cols = car_data.select_dtypes(include = 'number').columns

for col in numeric_cols:
  Q1 = car_data[col].quantile(0.25)
  Q3 = car_data[col].quantile(0.75)
  IQR = Q3-Q1
  lower = Q1 - 1.5 * IQR
  upper = Q3 + 1.5 * IQR
  car_data = car_data[(car_data[col]>=lower)&(car_data[col]<=upper)]

after_cleaning_rows = car_data.shape[0]

print(f"Before cleaning: {before_cleaning_rows} rows")
print(f"After cleaning: {after_cleaning_rows} rows")
print(f"Rows removed: {before_cleaning_rows - after_cleaning_rows}")

Before cleaning: 30652 rows
After cleaning: 14610 rows
Rows removed: 16042


### ii) Check for missing values

In [82]:
car_data.isnull().sum()

,0
ad_id,0
origin,0
condition,0
car_model,0
mileage,0
exterior_color,0
interior_color,0
num_of_doors,0
seating_capacity,0
engine,0


Recheck

In [83]:
original_row_count = car_data.shape[0]
before_cleaning_rows = original_row_count
after_cleaning_rows = car_data.shape[0]

print(f"Before cleaning: {before_cleaning_rows}")
print(f"After cleaning: {after_cleaning_rows}")
print(f"Rows removed: {before_cleaning_rows - after_cleaning_rows}")

Before cleaning: 14610
After cleaning: 14610
Rows removed: 0


# Data Storage

## a. Cars Table

In [89]:
df = car_data.rename(columns = {"price (AUD)":"price",})[["ad_id","car_name","origin","condition","exterior_color","car_model","year_of_manufacture","transmission","price","drive_type"]].copy()

conn = sqlite3.connect("car_database.db"); cur = conn.cursor()
cur.executescript("""
DROP TABLE IF EXISTS Cars;
CREATE TABLE Cars(
  ad_id INTEGER PRIMARY KEY,
  car_name VARCHAR (225),
  origin VARCHAR (225),
  condition VARCHAR (225),
  exterior_color VARCHAR (255),
  car_model VARCHAR (255),
  year_of_manufacture VARCHAR (255),
  transmission VARCHAR (255),
  price INTEGER,
  drive_type VARCHAR (255)
);
""")
df.to_sql("Cars",conn, if_exists="append", index=False)
conn.commit()

Results

In [91]:
pd.read_sql_query("SELECT * FROM Cars LIMIT 5", conn)

,ad_id,car_name,origin,condition,exterior_color,car_model,year_of_manufacture,transmission,price,drive_type
0,4664789,Toyota Yaris 1.5G 2016,imported,used car,white,hatchback,2016.0,automatic,17368.45973,fwd - front-wheel drive
1,4664930,Chevrolet Colorado LTZ 2.5L 4x4 AT 2019,imported,used car,yellow,4x4,2019.0,automatic,26923.13687,4wd - four-wheel drive (4wd)
2,4665042,Hyundai Tucson 2.0 ATH 2015,imported,used car,white,crossover,2015.0,automatic,26680.22135,fwd - front-wheel drive
3,4665290,Mazda CX5 2.0 AT 2017,domestic assembly,used car,white,suv,2017.0,automatic,26720.70727,fwd - front-wheel drive
4,4665362,VinFast Lux A 2.0 Advanced 2022 car,domestic assembly,new car,red,sedan,2022.0,automatic,33967.68697,rfd - rear-wheel drive


## b. Sellers Table

In [92]:
sellers = (pd.read_csv("/content/drive/MyDrive/3009ICT-Assignment/seller_dataset.csv", usecols = ["ad_id","name"]).rename(columns={"name":"seller_name"}))

conn = sqlite3.connect("car_database.db")
cur = conn.cursor()
cur.executescript("""
DROP TABLE IF EXISTS Sellers;
CREATE TABLE Sellers(
  ad_id INTEGER PRIMARY KEY,
  seller_name TEXT);""")
sellers.to_sql("Sellers", conn, if_exists="append", index=False)

30652

Results

In [93]:
display(pd.read_sql_query("SELECT * FROM Sellers LIMIT 5;", conn))
conn.close()

,ad_id,seller_name
0,17042,Suzuki - Sydney
1,53794,Toyota - Melbourne
2,73954,Toyota - Brisbane
3,74150,Toyota - Perth
4,87573,Toyota - Adelaide


## c. Query

In [100]:
conn = sqlite3.connect("car_database.db")

q = """
SELECT
  c.ad_id,
  c.car_name AS "Cars.car_name",
  c.price AS "Cars.Price",
  s.seller_name As "Sellers.seller_name"
  FROM Cars c
  JOIN Sellers s USING (ad_id)
  LIMIT 10;
  """
display(pd.read_sql_query(q, conn))
conn.close()

,ad_id,Cars.car_name,Cars.Price,Sellers.seller_name
0,4664789,Toyota Yaris 1.5G 2016,17368.45973,Toyota - Wollongong
1,4664930,Chevrolet Colorado LTZ 2.5L 4x4 AT 2019,26923.13687,Chevrolet - Geelong
2,4665042,Hyundai Tucson 2.0 ATH 2015,26680.22135,Hyundai - Townsville
3,4665290,Mazda CX5 2.0 AT 2017,26720.70727,Mazda - Burnie
4,4665362,VinFast Lux A 2.0 Advanced 2022 car,33967.68697,VinFast - Bunbury
5,4665368,VinFast Lux A 2.0 Advanced 2022 car,33967.68697,VinFast - Gladstone
6,4665404,VinFast Lux A 2.0 Premium 2022 Car,38056.76491,VinFast - Hervey Bay
7,4665490,Ford Ranger XLS 2.0L 4x2 MT 2023,25910.98887,Ford - Alice Springs
8,4665511,Ford Ranger XLS 2.0L 4x2 MT 2023,25910.98887,Ford - Mount Gambier
9,4665517,Ford Ranger XLS 2.0L 4x2 AT 2023,26720.70727,Ford - Kalgoorlie
